In [2]:
import sys

sys.path.append("C:\\Program Files (x86)\\MSCI Barra\\Barra Optimizer\\bin")
sys.path.append("C:\\Program Files (x86)\\MSCI Barra\\Barra Optimizer\\bin\\intel64")
sys.path.append("C:\\Program Files (x86)\\MSCI Barra\\Barra Optimizer\\tutorials\\python")

In [5]:
import barraopt
#!/usr/bin/env python
# A sample to call Barra Optimizer via SWIG C++ API

# Set the covariance matrix covData
covData = [
    [0.30, 0.12, 0.03],
    [0.12, 0.25, 0.18],
    [0.03, 0.18, 0.48],
    ]

# Set the specific covariance matrix speRisk
speRisk = [0.42, 0.64, 0.56, 0.49, 0.36]

# Set the exposure matrix expData
expData =[
    [0.4, 0.4, 0.2],
    [0.2, 0.7, 0.1],
    [0.1, 0.3, 0.6],
    [0.0, 1.0, 0.0],
    [0.5, 0.3, 0.2],
]
# Set the factors
factor = ['Factor_1A','Factor_1B','Factor_1C']

# Set the asset IDs
id = ['USABUY1','FRAAAC1','AUSANL1','USAANY1','UKIBEY1']

# Set the asset prices
price = [50.10, 35.95, 12.54, 74.25, 36.30]
price1 = [52, 34, 15, 80, 32]

# Set the asset alpha
alpha = [0.03, 0.11, 0.12, 0.08, 0.06]

# Set the managed portfolio weights
#mngWeight = [0.1, 0.3, 0.4, 0.1, 0.1]
mngWeight = [0.2, 0.2, 0.2, 0.2, 0.2]

# Set the benchmark portfolio weights
bmkWeight = [0.2, 0.2, 0.2, 0.2, 0.2]

# Set the managed portfolio name
mngName = 'ManagedPortfolio'

# Set the benchmark portfolio name
bmkName = 'BenchmarkPortfolio'

# Set constraints
#   Lower bounds
lB = [0.05,0.25,0.30,0.0,0.0]
#   Upper bounds
uB = [0.15,0.35,0.50,0.50,0.50]

# Constants
basevalue = 1000000.0
cashflowweight = 0.0


def Optimization (price, mngWeight):

    # Initialize java optimizer interface
    import barraopt

    # Create a workspace instance
    workspace = barraopt.CWorkSpace.CreateInstance()

    # Create a risk model
    rm = workspace.CreateRiskModel('SampleModel',barraopt.eEQUITY);

    # Add assets to workspace
    for i in range(len(id)):
        Asset = workspace.CreateAsset(id[i], barraopt.eREGULAR)    #CAsset
        Asset.SetAlpha(alpha[i]) 
        Asset.SetPrice(price1[i])   

    # Set the covariance matrix from data object into the workspace
    for i in range(len(factor)):
        for j in range(i,len(factor)):
            rm.SetFactorCovariance(factor[i], factor[j], covData[i][j])
            #print(i,j)

    # Set the exposure matrix from data object
    for i in range(len(id)):
        for j in range(len(factor)):
            rm.SetFactorExposure(id[i],factor[j],expData[i][j])

    # Set the specific riks covariance matrix from data object
    for i in range(len(id)):
        rm.SetSpecificCovariance(id[i], id[i], speRisk[i])

    # Create managed, benchmark and universe portfolio
    mngPortfolio = workspace.CreatePortfolio(mngName)
    bmkPortfolio = workspace.CreatePortfolio(bmkName)
    uniPortfolio = workspace.CreatePortfolio('universePortfolio')

    # Set weights to portfolio assets
    for i in range(len(id)):
        mngPortfolio.AddAsset(id[i], mngWeight[i])
        bmkPortfolio.AddAsset(id[i], mngWeight[i])
        uniPortfolio.AddAsset(id[i])  # no weight is needed for universe portfolio
    for i in range(len(id)):
        mngPortfolio.AddTaxLot(id[i],i**2,price1[i],basevalue*mngWeight[i]/price1[i],False)  #Age??




    # Create a case
    testcase = workspace.CreateCase('SampleCase', mngPortfolio, uniPortfolio, basevalue, cashflowweight)


    # Initialize constraints
    constr = testcase.InitConstraints()
    linearConstr = constr.InitLinearConstraints()
    for i in range(len(id)):
        info = linearConstr.SetAssetRange(id[i])
        info.SetLowerBound (lB[i])
        info.SetUpperBound (uB[i])



    # Set risk model & term 
    testcase.SetPrimaryRiskModel(workspace.GetRiskModel('SampleModel'));
    ut = testcase.InitUtility()
    ut.SetPrimaryRiskTerm(bmkPortfolio)

    #Tax Constraints
    tax = testcase.InitNewTax()     #CNewTax
    taxRule = tax.AddTaxRule('rule1','*')  #CTax
    taxRule.EnableTwoRate() # default is 365
    taxRule.SetTaxRate(0.243, 0.423) #long term and short term rates
    taxRule.SetWashSaleRule(barraopt.eDISALLOWED, 30)
    tax.SetSellingOrderRule('*','*',barraopt.eFIFO)




    solver = workspace.CreateSolver(testcase)

    # Uncomment the line below to dump workspace file
    #workspace.Serialize('opsdata.wsp')

    # Run optimizer
    status = solver.Optimize()

    if status.GetStatusCode() == barraopt.eOK: 
        # Optimization completed
        output = solver.GetPortfolioOutput();
        risk = output.GetRisk();
        utility = output.GetUtility();
        outputPortfolio = output.GetPortfolio();
        assetCount = outputPortfolio.GetAssetCount();

        outputWeight = [];
        OPTShares = []
        for i in range(len(id)):
            outputWeight.append(outputPortfolio.GetAssetWeight(id[i])); 

        # Show results on command window
        print('Optimization completed');
        print('Optimal portfolio risk: %g' % risk);
        print('Optimal portfolio utility: %g' % utility);
        for i in range(len(id)):
            print('Optimal number of shares of asset %s: %g'% (id[i], basevalue*outputWeight[i]/price[i]));
            OPTShares.append(basevalue*outputWeight[i]/price[i])
    else:
        # Optimization error
        print('Optimization error');


    taxOut = output.GetNewTaxOutput()
    newShares = taxOut.GetNewShares()

    initialShares = []
    
    for i in range(len(id)):
        initialShares.append(basevalue*mngWeight[i]/price[i])
        
    for i in range(len(id)):
        print('Original number of shares of asset %s: %g'% (id[i],initialShares[i]))
        
        
    for i in range(len(id)):
        sharesInTaxlot=taxOut.GetSharesInTaxLots(id[i])
        for lotID,shares in sharesInTaxlot.items():
            if shares!=0:
                print('Tax lot of asset %s: %g'% (lotID, shares))
                
                
    for asset,newshares in newShares.items():
        print('New number of shares of asset %s: %g'% (asset,newshares))
        
        
    return initialShares, newShares, outputWeight, OPTShares

    

    
    
#workspace.Release();



In [7]:


# Initialize java optimizer interface
import barraopt

# Create a workspace instance
workspace = barraopt.CWorkSpace.CreateInstance()

# Create a risk model
rm = workspace.CreateRiskModel('SampleModel',barraopt.eEQUITY);

# Add assets to workspace
for i in range(len(id)):
    Asset = workspace.CreateAsset(id[i], barraopt.eREGULAR)    #CAsset
    Asset.SetAlpha(alpha[i]) 
    Asset.SetPrice(price1[i])   

# Set the covariance matrix from data object into the workspace
for i in range(len(factor)):
    for j in range(i,len(factor)):
        rm.SetFactorCovariance(factor[i], factor[j], covData[i][j])
        #print(i,j)

# Set the exposure matrix from data object
for i in range(len(id)):
    for j in range(len(factor)):
        rm.SetFactorExposure(id[i],factor[j],expData[i][j])

# Set the specific riks covariance matrix from data object
for i in range(len(id)):
    rm.SetSpecificCovariance(id[i], id[i], speRisk[i])

# Create managed, benchmark and universe portfolio
mngPortfolio = workspace.CreatePortfolio(mngName)
bmkPortfolio = workspace.CreatePortfolio(bmkName)
uniPortfolio = workspace.CreatePortfolio('universePortfolio')

# Set weights to portfolio assets
for i in range(len(id)):
    mngPortfolio.AddAsset(id[i], mngWeight[i])
    bmkPortfolio.AddAsset(id[i], mngWeight[i])
    uniPortfolio.AddAsset(id[i])  # no weight is needed for universe portfolio
for i in range(len(id)):
    mngPortfolio.AddTaxLot(id[i],i**2,price1[i],basevalue*mngWeight[i]/price1[i],False)  #Age??




# Create a case
testcase = workspace.CreateCase('SampleCase', mngPortfolio, uniPortfolio, basevalue, cashflowweight)


# Initialize constraints
constr = testcase.InitConstraints()
linearConstr = constr.InitLinearConstraints()
for i in range(len(id)):
    info = linearConstr.SetAssetRange(id[i])
    info.SetLowerBound (lB[i])
    info.SetUpperBound (uB[i])



# Set risk model & term 
testcase.SetPrimaryRiskModel(workspace.GetRiskModel('SampleModel'));
ut = testcase.InitUtility()
ut.SetPrimaryRiskTerm(bmkPortfolio)

#Tax Constraints
tax = testcase.InitNewTax()     #CNewTax
taxRule = tax.AddTaxRule('rule1','*')  #CTax
taxRule.EnableTwoRate() # default is 365
taxRule.SetTaxRate(0.243, 0.423) #long term and short term rates
taxRule.SetWashSaleRule(barraopt.eDISALLOWED, 30)
tax.SetSellingOrderRule('*','*',barraopt.eFIFO)




solver = workspace.CreateSolver(testcase)

# Uncomment the line below to dump workspace file
#workspace.Serialize('opsdata.wsp')

# Run optimizer
status = solver.Optimize()

if status.GetStatusCode() == barraopt.eOK: 
    # Optimization completed
    output = solver.GetPortfolioOutput();
    risk = output.GetRisk();
    utility = output.GetUtility();
    outputPortfolio = output.GetPortfolio();
    assetCount = outputPortfolio.GetAssetCount();

    outputWeight = [];
    OPTShares = []
    for i in range(len(id)):
        outputWeight.append(outputPortfolio.GetAssetWeight(id[i])); 

    # Show results on command window
    print('Optimization completed');
    print('Optimal portfolio risk: %g' % risk);
    print('Optimal portfolio utility: %g' % utility);
    for i in range(len(id)):
        print('Optimal number of shares of asset %s: %g'% (id[i], basevalue*outputWeight[i]/price[i]));
        OPTShares.append(basevalue*outputWeight[i]/price[i])
else:
    # Optimization error
    print('Optimization error');


taxOut = output.GetNewTaxOutput()
newShares = taxOut.GetNewShares()

initialShares = []

for i in range(len(id)):
    initialShares.append(basevalue*mngWeight[i]/price[i])

for i in range(len(id)):
    print('Original number of shares of asset %s: %g'% (id[i],initialShares[i]))


for i in range(len(id)):
    sharesInTaxlot=taxOut.GetSharesInTaxLots(id[i])
    for lotID,shares in sharesInTaxlot.items():
        if shares!=0:
            print('Tax lot of asset %s: %g'% (lotID, shares))


for asset,newshares in newShares.items():
    print('New number of shares of asset %s: %g'% (asset,newshares))

        



Optimization completed
Optimal portfolio risk: 11.1948
Optimal portfolio utility: 0.0815308
Optimal number of shares of asset USABUY1: 2192.14
Optimal number of shares of asset FRAAAC1: 6954.1
Optimal number of shares of asset AUSANL1: 23923.4
Optimal number of shares of asset USAANY1: 2508.33
Optimal number of shares of asset UKIBEY1: 4240.5
Original number of shares of asset USABUY1: 3992.02
Original number of shares of asset FRAAAC1: 5563.28
Original number of shares of asset AUSANL1: 15949
Original number of shares of asset USAANY1: 2693.6
Original number of shares of asset UKIBEY1: 5509.64
New number of shares of asset AUSANL1: 6666.67
New number of shares of asset FRAAAC1: 1470.59


In [16]:
mngPortfolio.AddWashSaleRec('USABUY1', 1, 20, 100, False)

'USABUY1_WashSaleLot_1'

In [17]:
a = taxOut.GetWashSaleDetail('USABUY1')
print(a)

None


In [ ]:
# tw0 Period Optimization- Cumulative Tax

In [122]:
Sell_Lot = {}
Buy_Lot = {}
Total_Sell = 0
basevalue = 1000000
Withdraw_amount = 50000
Invest_amount = 50000 
mngWeight = [0.2, 0.2, 0.2, 0.2, 0.2]


id = ['USABUY1','FRAAAC1','AUSANL1','USAANY1','UKIBEY1']
price_1 = [50.10, 35.95, 12.54, 74.25, 36.30]  
price_2 = [52, 34, 15, 80, 32]  #suppose withdraw
price_31 = [52, 34, 15, 80, 32]  #suppose invest

# Day 1
init, adj, wgt, OPT = Optimization(day_1_price, day_1_management_weight)
management_weight = wgt

# Day 2
basevalue -= Withdraw_amount
init2, adj2, wgt2, OPT2 = Optimization(day_1_price, management_weight)


# Set Buy/ Sell Stocks
Buy = {}
Sell = {}
for i in range(len(id)):
    gap = OPT2[i] - OPT[i]
    if gap > 0:
        Buy[id[i]] = gap
        

    else:
        Sell[id[i]] = -gap
        
        Total_Sell += -gap * price_2[i]
        

        
    
# Day 31  (Buy Back Sell Lot)
basevalue += Invest_amount
management_weight = wgt2 
init31, adj31, wgt31, OPT31 = Optimization(day_31_price, management_weight)

# Set Buy/ Sell Stocks

for i in range(len(id)):
    gap = OPT31[i] - OPT31[i]
    if gap > 0:
        if id[i] not in Sell.keys():
            Buy[id[i]] = gap
            
        else:
            if gap < Sell[id[i]]:
                Buy_amount = gap
                Sell[id[i]] -= gap
                
                
                
            else:
                Buy_amount = gap
                del Sell[id[i]]
                
            
    else:
        if id[i] not in Sell.keys():
            Sell[id[i]] = -gap
            Total_Sell += -gap * price_31[i]
        else:
            Sell[id[i]] += -gap
            Total_Sell += -gap * price_31[i]
    





Optimization completed
Optimal portfolio risk: 11.1948
Optimal portfolio utility: 0.0815308
Optimal number of shares of asset USABUY1: 2192.14
Optimal number of shares of asset FRAAAC1: 6954.1
Optimal number of shares of asset AUSANL1: 23923.4
Optimal number of shares of asset USAANY1: 2508.33
Optimal number of shares of asset UKIBEY1: 4240.5
Original number of shares of asset USABUY1: 3992.02
Original number of shares of asset FRAAAC1: 5563.28
Original number of shares of asset AUSANL1: 15949
Original number of shares of asset USAANY1: 2693.6
Original number of shares of asset UKIBEY1: 5509.64
New number of shares of asset AUSANL1: 6666.67
New number of shares of asset FRAAAC1: 1470.59
Optimization completed
Optimal portfolio risk: 6.30897
Optimal portfolio utility: 0.094324
Optimal number of shares of asset USABUY1: 948.104
Optimal number of shares of asset FRAAAC1: 7541.25
Optimal number of shares of asset AUSANL1: 26198.2
Optimal number of shares of asset USAANY1: 2425.48
Optimal n

In [124]:
Tax = Total_Sell * 0.24  #shorterm

Tax

23707.938941217217